# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (D - A) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [2]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [3]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [4]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [5]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [6]:
# Check the result:

A @ x - b

array([ 0.00000000e+00,  2.22044605e-16,  0.00000000e+00, -1.11022302e-16,
        0.00000000e+00,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
        0.00000000e+00,  2.22044605e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [7]:
def jacobi_iter(A, b, n):
    diag = np.diag(A)
    B = np.diag(1. / diag) @ (np.diag(diag) - A)
    print('Норма матицы B равна {}\n'.format(np.linalg.norm(B)))
    c = np.diag(1. / diag) @ b
    x = np.ones(b.shape)
    for i in range(n):
        x = B @ x + c
    return x

In [8]:
n1 = 8
A1 = rndm.uniform(size=(n1, n1)) + np.diagflat([15]*n1)
b1 = rndm.uniform(size=n1)

In [9]:
x1 = jacobi_iter(A1, b1, 30)

print(A1 @ x1 - b1)

Норма матицы B равна 0.29417978724692306

[ 0.00000000e+00 -1.11022302e-16  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -1.11022302e-16 -1.11022302e-16  0.00000000e+00]


Как видно, в случае, когда матрица $A_1$ диагонально доминирующая, алгоритм хорошо сходится к правильному решению. Теперь заменим диагональные элементы маттрицы на её минимальный элемент

In [10]:
np.fill_diagonal(A1, A1.min())

x1_1 = jacobi_iter(A1, b1, 30)
print(A1 @ x1_1 - b1)

Норма матицы B равна 492.00142565985897

[1.90466166e+78 2.52675686e+78 2.12003664e+78 1.94302934e+78
 1.83504824e+78 2.75968872e+78 2.89982295e+78 2.44239878e+78]


В этом случае $\| B\| \gg 1$, и, как следствие, алгоритм не сходится к правильному ответу

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

$$Ax=b$$
$$A=L+D+U$$
$$\Rightarrow x^{(k+1)}=-(L+D)^{-1}\cdot U\cdot x^{(k)}+(L+D)^{-1}\cdot b$$

Сначала напишем функцию, которая будет считать $(L+D)^{-1}$, то есть функцию, которая находит обратную матрицу к нижнетреугольной матрице*

*необходимо, чтобы на главной диагонале у первоначальной матрицы не было нулей

In [11]:
def inverse(A):
    n = A.shape[0]
    inv = np.diag(1 / np.diag(A))
    e = inv @ A
    for i in range(n - 1):
        k = np.eye(n)
        k[i+1:,i] = -e[i+1:,i]
        e = k @ e
        inv = k @ inv
    return inv

Теперь сам метод Зейделя:

In [12]:
def seidel_iter(A, b, t):
    n = b.shape[0]
    x = np.ones_like(b)
    U = np.zeros_like(A)
    LD = np.zeros_like(A)
    for i in range(n):
        U[:i,i] = A[:i,i]
        LD[i:,i] = A[i:,i]
    LD_inv = inverse(LD)
    print('Норма матицы (L+D)^-1 равна {}\n'.format(np.linalg.norm(LD_inv)))
    for i in range(t):
        x = -LD_inv @ U @ x + LD_inv @ b
    return x

Сначала проверим работу алгоритма на диагонально доминирующей матрице

In [13]:
n2 = 10
A2 = rndm.uniform(size=(n2, n2)) + np.diagflat([15]*n2)
b2 = rndm.uniform(size=n2)

In [14]:
x2 = seidel_iter(A2, b2, 20)

print(A2 @ x2 - b2)

Норма матицы (L+D)^-1 равна 0.20477939937010078

[-1.11022302e-16  0.00000000e+00  1.11022302e-16  0.00000000e+00
 -1.11022302e-16 -1.11022302e-16  0.00000000e+00 -5.55111512e-17
  0.00000000e+00  0.00000000e+00]


Как видно, в этом случае алгоритм хорошо сходится к правильному результату

Теперь проверим алгоритм на произвольной матрице

In [15]:
n3 = 10
A3 = rndm.uniform(size=(n3, n3))
b3 = rndm.uniform(size=n3)

In [16]:
x3 = seidel_iter(A3, b3, 20)

print(A3 @ x3 - b3)

Норма матицы (L+D)^-1 равна 846.2518782201703

[ 1.04120715e+53  7.32382284e+51  5.53691734e+52  6.00573842e+52
  8.21709195e+52  8.98023193e+52  3.27438629e+52  3.56989409e+52
  2.88502955e+52 -1.16307450e+36]


Как и ожидалось, при $\| (L+D)^{-1}\| > 1$ метод Зейделя также не сходится к правильному ответу

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...